In [1]:
# Code to import 
import pymongo
import requests
from pprint import pprint

# Functions

In [2]:
# A function to do an api request, then convert the data to json format
def api_request(url):
    raw_data = requests.get(url)
    json_data = raw_data.json()
    return json_data

# A function to turn the pages 
def turn_page(json_data):
    if json_data["next"] != "null":
        next_page_url = json_data["next"]
        return next_page_url
    else:
        return False

# A function to call the url from all of the pages
def url_in_api(page_url_info):
    url_list_ships = []  # A list of all starship url
    for url in page_url_info:
        data = api_request(url)
        for i in data["results"]:
            url_list_ships.append(i["url"])
    return url_list_ships

# A function to create a list and append the pilot url
def pilot_url_list(url_list_ship_info):
    ship_pilot_url = []
    for i in url_list_ship_info:
        ship_info = api_request(i)
        pilot_url = ship_info["result"]["properties"]["pilots"]
        ship_pilot_url.append(pilot_url)
    return ship_pilot_url

# A function to flatten the lists of lists.
def flatten(ship_pilot_url):
    pilot_urls_flat = []
    for sublist in ship_pilot_url:
        for item in sublist:
            pilot_urls_flat.append(item)
    return pilot_urls_flat

# A function to create a list with just the names 
def name(pilot_urls_flat):
    pilot_name_list = []
    for i in pilot_urls_flat:
        people_info = api_request(i)
        pilot_id = people_info["result"]["properties"]["name"]
        pilot_name_list.append(pilot_id)
    return pilot_name_list

# A function to return a list of just the object id by comparing the names
def mongo_name(pilot_name_list):
    mongo_name_id_list = []
    for name in pilot_name_list:
        mongo_name_id = db.characters.find_one({"name": name}, {"_id": 1})
        mongo_name_id_list.append(mongo_name_id)
    return mongo_name_id_list

# A function to make a dictionary
def make_dict(key, value):
    new_dict = {k: v for k, v in zip(key, value)}
    return new_dict

# Code - main in pycharm

In [3]:
# Code to use mongo, access the satabase starwars and drop/make a collection called starships
client = pymongo.MongoClient()
db = client["starwars"]
db.drop_collection("starships")  # Drops collection if exists
db.create_collection("starships")

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'starwars'), 'starships')

In [4]:
# Name the api, add first page of x to a list.
api_address_ship = "https://www.swapi.tech/api/starships"
web_address = api_address_ship
page_url = [api_address_ship]

In [5]:
# Calls two functions, api request and turn_page. Pages added to page_url list, breaks if next page = None
while True:
    page_contents = api_request(web_address)
    next_page_url = turn_page(page_contents)
    if next_page_url == None:
        break
    else:
        page_url.append(next_page_url)
        web_address = next_page_url

In [6]:
# A list of the ship url using the pages - calls function url_in_api
url_list_ship = url_in_api(page_url)

In [7]:
# A list of the pilot urls from each ship, only returns ships with pilots, removes empty lists
ship_pilot_url = pilot_url_list(url_list_ship)
ship_pilot_url = [x for x in ship_pilot_url if x]

In [8]:
# flattens the data, removes a list of lists
pilot_urls_flat = flatten(ship_pilot_url)

In [9]:
# a list of just the pilot names from the flat list
pilot_name_list = name(pilot_urls_flat)

In [10]:
# A list of ids corresponding to the name, by matching the "pilot name" with the name in the starwars database
mongo_name_id_list = mongo_name(pilot_name_list)

In [11]:
# A dictionary with all the pilots url and pilot names
key_list = pilot_urls_flat
value_list = mongo_name_id_list
pilot_url_mongo_name_id = make_dict(key_list, value_list)

In [12]:
# Code to insert the objectid to the starships collection with 0, 1 or multiple pilotIDs
for url in url_list_ship:
    ship_data = api_request(url)
    people_url_id_list = []
    for p_url in ship_data["result"]["properties"]["pilots"]:
        if p_url == []:
            pass
        else:
            people_url_id_list.append(pilot_url_mongo_name_id[p_url])
            ship_data["result"]["properties"]["pilots"] = people_url_id_list
    db.starships.insert_one(ship_data)


# Testing


In [13]:
import requests

# This code test that the output is a dictionary
def func_one_test(url):
    assert type(api_request(url)) is int

# This code tests the status code is 200
def func_two_test(url):
    check = api_request(url)
    assert check.status_code == 200

# This tests the length of the list is equal to the total number of pages
def func_three_test(list, data):
    assert len(list) == data["total_pages"]

# Additional Comments

This is the main submit of the code, api issues persisted with a 5minute interval between runs. 